# Download European Country Shapefiles

Very unfortunately, there is no simple list of these shapefiles, but rather a GUI formatted frontend that lists all countries, and a series of pages that direct to country specific download links.

We need to employ BeautifulSoup, luckily, the page html is static.

In [6]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json
import zipfile
from pathlib import Path
import io
import os
import pandas as pd
import time
from urllib.parse import urlparse

In [28]:
ROOT = "/project_data/data_asset/sun/geo"
Path(ROOT).mkdir(parents=True,exist_ok=True)

catalogue = []

for country in tqdm(["DEU","GBR","FRA","AUT"]):
    url = "https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_{}_shp.zip".format(country)
    bits = urlparse(url)
    path,filename = os.path.split(bits.path)
    if Path(os.path.join(ROOT,filename)).is_file():
        print("Already have "+os.path.join(ROOT,filename))
        continue
    r = requests.get(url)
    if not r.ok:
        print("Cannot download {}".format(country))
    with open(os.path.join(ROOT,filename),"w+b") as outfile:
        outfile.write(r.content)
        
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    newROOT = os.path.join(ROOT,country)
    Path(newROOT).mkdir(parents=True,exist_ok=True)
    
    zf.extractall(newROOT)
    
    for zf in zf.filelist:
        catalogue.append({"country":country,"url":url,
                         "path":newROOT,"filename":zf.filename,"fullpath":os.path.join(newROOT,zf.filename),
                         "extension":os.path.splitext(zf.filename)[1],"size":zf.file_size})
    pd.DataFrame(catalogue).to_csv(os.path.join(ROOT,"catalogue.csv"),index=False)
    
    time.sleep(10)

100%|██████████| 4/4 [00:19<00:00,  4.95s/it]

Already have /project_data/data_asset/sun/geo/gadm36_GBR_shp.zip
Already have /project_data/data_asset/sun/geo/gadm36_FRA_shp.zip
Already have /project_data/data_asset/sun/geo/gadm36_AUT_shp.zip


In [25]:
r.ok

True